In [1]:
import pandas as pd
import numpy as np

### Catalog cleaning data

In [2]:
# import catalog data 
df_cat = pd.read_csv('/Users/Mary/Desktop/NLP/Group Project/adbl_catalog_cleaned.csv')
del df_cat["Unnamed: 0"] # delete unnecessary index column saved from scraping

In [3]:
df_cat.head()

,product_asin,product_short_title,product_author,release_date,rating,rating_count,legacy_genre
0,B07S64NL7C,Overcoming Anger (Unabridged),Bubba Hemp,2019-05-23,3.5,2,Education & Learning
1,B00PMESE5C,Mouth Watering Super Shred Diet Recipes (Unabr...,Alex Grayson,2014-11-17,2.0,1,Health & Wellness
2,B09TG64J8C,Elite Productivity (Unabridged),BAB Group,2022-02-28,0.0,0,"Relationships, Parenting & Personal Development"
3,B08FRK6G4M,The Tatars (Unabridged),Charles River Editors,2020-08-12,5.0,1,History
4,B07YGX7YGJ,1001 Questions to Help Flesh Out Your Characte...,Christina Escamilla,2019-09-28,0.0,0,NaN


In [4]:
# clean titles, make lowercase, removing things like 'unabridged', and strip, punctuation, extra whitespace
df_cat['product_short_title'] = df_cat['product_short_title'].str.lower()
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\(unabridged\)", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\(abridged\)", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 1", "").str.strip()
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 2", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 3", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 4", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 5", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 6", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 7", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 8", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 9", "").str.strip() 
df_cat["product_short_title"] = df_cat["product_short_title"].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)


/var/folders/cv/l7wnv7t94vvfyk2t65yjhyy80000gn/T/ipykernel_8430/3045902568.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\(unabridged\)", "").str.strip()
/var/folders/cv/l7wnv7t94vvfyk2t65yjhyy80000gn/T/ipykernel_8430/3045902568.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\(abridged\)", "").str.strip()
/var/folders/cv/l7wnv7t94vvfyk2t65yjhyy80000gn/T/ipykernel_8430/3045902568.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\, book 1", "").str.strip()
/var/folders/cv/l7wnv7t94vvfyk2t65yjhyy80000gn/T/ipykernel_8430/3045902568.py:6: FutureWarning: The default value of regex will change from True to Fals

In [5]:
# create list of titles that are the exception to "one word title exclusion" rule 

one_word_exceptions = ['1797113461','1713570874','1789999103','B08VYFG11M','B08S622QNT','1800263686','B08V4MTZTP','1526622416','B08YFM1565','593459296','593212142','1529342015','593168070','B08B46CK1J','B08J8BJV3T','B082MQ5TDB','1774243326','B086WNG3RK','B086WMZ9WR','1774246325','593294181']


### add a whitespace count to titles and remove one word titles that were:
1. released before April 2020 (not released in last 2 years)
2. have fewer than 1K ratings (assume this is indicative of low popularity/sales)

In [9]:
# remove titles with length less than or equal to 3 characters (this removes titles like "Y" "X" "Her") 
# titles like this are too difficult to identify via string matching, we'd rather underassign tweets that are about 
# these titles than overassin tweets that don't contain titles but do contain the word "her" or letter "y"
import datetime

# set date to 2 years ago and convert release date column to datetime
date_before = datetime.datetime(2020, 4, 1)
df_cat['release_date'] = pd.to_datetime(df_cat['release_date'])

# add a column that counts number of spaces in title 
df_cat['whitespace_ct'] = df_cat['product_short_title'].str.count(' ')

# print catalog size before dropping records
print(str(len(df_cat))+' titles before dropping records') 

# remove titles < 3 characters
df_cat.drop(df_cat[df_cat['product_short_title'].map(len) <= 3].index, inplace=True)

# keep only titles with at least 1 space (i.e. 2 words) or that appear in the exception list
df_cat = df_cat.loc[((df_cat['whitespace_ct'] > 0) | (df_cat['product_asin'].isin(one_word_exceptions)))]

# drop titles with fewer than 10 ratings - consider this a proxy for sales 
df_cat.drop(df_cat[df_cat['rating_count'] < 10].index, inplace=True)

# print catalog size before dropping records

print(str(len(df_cat))+' titles after dropping records') 


517524 titles before dropping records
203319 titles after dropping records


In [10]:
# sort catalog df by # of ratings - enteries are ordered in the dictionary by order of insertion
# this ensures that for titles that exist as substrings of other titles (ex: Raw is contained in Where the Crawdad's Sing)
# the most popular title (based on rating count) will get assigned to a tweet first 
df_cat = df_cat.sort_values(by=['rating_count'],ascending=False)

In [11]:
# duplicate title column removing spaces
# this will allow us to identify titles that may have been used as hashtags rather than fully named in the tweet
df_cat["title_no_spaces"] = df_cat["product_short_title"].str.replace(" ", "").str.strip() 

In [12]:
df_cat.head()

,product_asin,product_short_title,product_author,release_date,rating,rating_count,legacy_genre,whitespace_ct,title_no_spaces
149211,B07FSNSLZ1,where the crawdads sing,Delia Owens,2018-08-14,4.760701,193799,Romance,3,wherethecrawdadssing
28988,B01I28NFEE,the subtle art of not giving a f*ck,Mark Manson,2016-09-13,4.516414,144510,"Relationships, Parenting & Personal Development",7,thesubtleartofnotgivingaf*ck
308431,B07KKMNZCH,can't hurt me,David Goggins,2018-11-28,4.874660,139548,"Relationships, Parenting & Personal Development",2,can'thurtme
445593,B017V4IM1G,harry potter and the sorcerer's stone,J.K. Rowling,2015-11-20,4.903521,139222,Science Fiction & Fantasy,5,harrypotterandthesorcerer'sstone
159139,B002UZZ93G,a game of thrones,George R.R. Martin,2003-12-09,4.757246,128418,Literature & Fiction,3,agameofthrones


In [13]:
# merge both title columns into a list, this list will become the value in the catalog dictionary
df_cat["both_titles"] = list(zip(df_cat['product_short_title'], df_cat['title_no_spaces']))

In [14]:
# convert the catalog asin (unique identifier) & title list into a dictionary 
cat_dict = df_cat.set_index('product_asin').to_dict()['both_titles']

In [15]:
cat_dict

{'B07FSNSLZ1': ('where the crawdads sing', 'wherethecrawdadssing'),
 'B01I28NFEE': ('the subtle art of not giving a f*ck',
  'thesubtleartofnotgivingaf*ck'),
 'B07KKMNZCH': ("can't hurt me", "can'thurtme"),
 'B017V4IM1G': ("harry potter and the sorcerer's stone",
  "harrypotterandthesorcerer'sstone"),
 'B002UZZ93G': ('a game of thrones', 'agameofthrones'),
 'B017V4IWVG': ('harry potter and the chamber of secrets',
  'harrypotterandthechamberofsecrets'),
 'B002UZMLXM': ('the name of the wind', 'thenameofthewind'),
 'B017V4JA2Q': ('harry potter and the prisoner of azkaban',
  'harrypotterandtheprisonerofazkaban'),
 'B017V4NUPO': ('harry potter and the goblet of fire',
  'harrypotterandthegobletoffire'),
 'B003ZWFO7E': ('the way of kings', 'thewayofkings'),
 'B002UZKIBO': ('a clash of kings', 'aclashofkings'),
 'B017V4NMX4': ('harry potter and the order of the phoenix',
  'harrypotterandtheorderofthephoenix'),
 'B06Y1GBY2M': ('before we were yours', 'beforewewereyours'),
 'B0731R4RFX': ('

### title to tweet function

In [41]:
# define function that loops through dictionary and tweetes to check if title exists in tweet
def getASIN(x):
    for key, values in cat_dict.items():
        for value in values:
            if value in x:
                return key
    else:
        return np.nan

#### test the function on a small mock dataset to ensure it works properly

In [42]:
# load test tweets
# running function against a small sample of fake tweets to confirm it works appropriately 
test_df = pd.read_csv('/Users/Mary/Desktop/NLP/Group Project/test tweets.csv')

In [43]:
test_df.head()

,Datetime,Tweet Id,Text,Username,hashtag
0,2022-03-23 23:50:07+00:00,1.506780e+18,"where the crawdads sing was a great book, name...",actuallydana,#booktok
1,2022-03-23 23:15:05+00:00,1.506770e+18,i love reading can't hurt me,IAN_AuthorPromo,#booktok
2,2022-03-23 23:00:01+00:00,1.506770e+18,"reading the name of the wind right now, so good",LovinSunshine97,#booktok
3,2022-03-23 23:00:01+00:00,1.506770e+18,xxxxxxxxx,LovinSunshine97,#booktok
4,2022-03-23 23:00:01+00:00,1.506770e+18,reading #agameofthrones,LovinSunshine97,#booktok


In [44]:
tweets = test_df['Text'].tolist()

In [45]:
tweets

['where the crawdads sing was a great book, name of the wind is also good',
 "i love reading can't hurt me",
 'reading the name of the wind right now, so good',
 'xxxxxxxxx',
 'reading #agameofthrones',
 'I love reading so much']

In [46]:
[key for key, value in cat_dict.items() if value in tweets]

[]

In [47]:
test_df['ASIN'] = test_df['Text'].apply(getASIN)

In [48]:
test_df.head(10)
# crawdads = B07FSNSLZ1
# can't hurt me = B07KKMNZCH
# name of the wind = B002UZMLXM

,Datetime,Tweet Id,Text,Username,hashtag,ASIN
0,2022-03-23 23:50:07+00:00,1.506780e+18,"where the crawdads sing was a great book, name...",actuallydana,#booktok,B07FSNSLZ1
1,2022-03-23 23:15:05+00:00,1.506770e+18,i love reading can't hurt me,IAN_AuthorPromo,#booktok,B07KKMNZCH
2,2022-03-23 23:00:01+00:00,1.506770e+18,"reading the name of the wind right now, so good",LovinSunshine97,#booktok,B002UZMLXM
3,2022-03-23 23:00:01+00:00,1.506770e+18,xxxxxxxxx,LovinSunshine97,#booktok,NaN
4,2022-03-23 23:00:01+00:00,1.506770e+18,reading #agameofthrones,LovinSunshine97,#booktok,B002UZZ93G
5,2022-03-23 23:00:01+00:00,1.506770e+18,I love reading so much,LovinSunshine97,#booktok,NaN


In [53]:
cat_dict.get('B002UZZ93G')

('a game of thrones', 'agameofthrones')

### load & cleaning tweet data

In [54]:
df_tweets = pd.read_csv('/Users/Mary/Desktop/NLP/Group Project/full_tweet_set.csv')

/Users/Mary/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
df_tweets['Text'] = df_tweets['Text'].astype('string')
df_tweets['Tweet Id'] = df_tweets['Tweet Id'].astype('string')
df_tweets['Username'] = df_tweets['Username'].astype('string')
df_tweets['hashtag'] = df_tweets['hashtag'].astype('string')


In [56]:
print(len(df_tweets))
df_tweets.drop_duplicates(subset='Tweet Id',inplace=True)
print(len(df_tweets))

168435948
1276598


In [57]:
print(len(df_tweets))
df_tweets.dropna(subset=['Text'],inplace=True)
print(len(df_tweets))


1276598
1276597


In [58]:
df_tweets.head()

,Datetime,Tweet Id,Text,Username,hashtag
0,2022-03-23 23:50:07+00:00,1506780360834748419,Off to a great start #TheSevenHusbandsOfEvely...,actuallydana,#booktok
1,2022-03-23 23:15:05+00:00,1506771544600719367,"Enoch by Jarrod Edge #christian #scifi ""Prophe...",IAN_AuthorPromo,#booktok
2,2022-03-23 23:00:01+00:00,1506767752152985610,"Soooooo, I’m on 📷TikTok: https://t.co/DyaEldN1...",LovinSunshine97,#booktok
3,2022-03-23 22:52:17+00:00,1506765808218820610,GUYS! I just hit 2K on my BookTok account 😭💖💖 ...,adorkablebooks,#booktok
4,2022-03-23 21:44:13+00:00,1506748679109160969,Thank you to @UnionSqandCo for believing in Ke...,themelissablair,#booktok


In [60]:
# make tweets lowercase for string matching
df_tweets['Text'] = df_tweets['Text'].str.lower() #.astype('unicode')


In [61]:
df_tweets_sample = df_tweets.head(100)

### run the title to tweet function over tweet dataset

In [62]:
df_tweets_sample['ASIN'] = df_tweets_sample['Text'].apply(getASIN)

/var/folders/cv/l7wnv7t94vvfyk2t65yjhyy80000gn/T/ipykernel_8430/1168861302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_sample['ASIN'] = df_tweets_sample['Text'].apply(getASIN)


In [63]:
df_tweets_sample.head()

,Datetime,Tweet Id,Text,Username,hashtag,ASIN
0,2022-03-23 23:50:07+00:00,1506780360834748419,off to a great start #thesevenhusbandsofevely...,actuallydana,#booktok,B072359S7K
1,2022-03-23 23:15:05+00:00,1506771544600719367,"enoch by jarrod edge #christian #scifi ""prophe...",IAN_AuthorPromo,#booktok,B0064RTUO0
2,2022-03-23 23:00:01+00:00,1506767752152985610,"soooooo, i’m on 📷tiktok: https://t.co/dyaeldn1...",LovinSunshine97,#booktok,B07L5YVGDC
3,2022-03-23 22:52:17+00:00,1506765808218820610,guys! i just hit 2k on my booktok account 😭💖💖 ...,adorkablebooks,#booktok,NaN
4,2022-03-23 21:44:13+00:00,1506748679109160969,thank you to @unionsqandco for believing in ke...,themelissablair,#booktok,B002V1POWK


In [ ]:
df_tweets['ASIN'] = df_tweets['Text'].apply(getASIN)

In [ ]:
df_tweets.head()

In [ ]:
# count number of tweets with no ASIN assigned 
# i.e. do not contain a title 
df_tweets['ASIN'].isna().sum()

In [ ]:
df_tweets.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
df_tweets[:1]['Text']

In [ ]:
df_tweets.to_csv('/Users/Mary/Desktop/NLP/Group Project/tweets_with_asin.csv')

In [ ]:
df_tweets_no_null = df_tweets.dropna(subset=['ASIN'])

In [ ]:
len(df_tweets_no_null)

In [ ]:
# group tweets by asin & count number of occurrences 
asin_cts = df_tweets_no_null.groupby(['ASIN']).size().reset_index()

In [ ]:
# sort for top 10 asins appearing in tweets
asin_cts.sort_values(0, ascending=False).head()

In [ ]:
cat_dict.get('B0048G19IE')

In [ ]:
df_tweets.dtypes

In [ ]:
df_tweets.head()